In [42]:
# !pip install requests

In [43]:
import requests
import json
import os
import glob
from dotenv import load_dotenv

load_dotenv()
gov_service_key = os.getenv('GOV_SERVICE_KEY')

In [44]:
# API 요청 설정
# API의 기본 URL
url = "http://apis.data.go.kr/B552555/lhLeaseNoticeInfo1/lhLeaseNoticeInfo1"

# API에 전달할 파라미터 (딕셔너리 형태)
params = {
    'serviceKey': gov_service_key,  # .env에서 불러온 인증키
    'PG_SZ': '10',                 # 한 페이지 결과 수
    'PAGE': '1',                    # 페이지 번호
    'PAN_NM': '서울',             # 공고명으로 검색 (예: 대전, 서울 등)
    'UPP_AIS_TP_CD': '01',        # 공고유형코드
    'CNP_CD': '11',               # 지역코드 (11=서울, 26=부산, 27=대구, 28=인천, 29=광주, 30=대전, 31=울산 등)
    'PAN_SS': '공고중',            # 공고상태 (공고중, 마감 등)
    'PAN_NT_ST_DT': '20250101',   # 공고게시 시작일 (YYYYMMDD)
    'CLSG_DT': '20251111',        # 공고마감일 (YYYYMMDD)
}

# 키가 제대로 로드되었는지 확인
if not gov_service_key:
    print("오류: .env 파일에서 'GOV_SERVICE_KEY'를 찾을 수 없거나 값이 비어있습니다.")
else:
    print("서비스 키 로드 완료")

서비스 키 로드 완료


In [45]:
# --- API 호출 및 응답 처리 ---
try:
    # requests가 'params' 딕셔너리를 URL 파라미터(?key=value&...)로 자동 변환
    response = requests.get(url, params=params)

    # HTTP 상태 코드가 200(성공)이 아니면 오류 발생
    response.raise_for_status()

    print("API 호출 성공")

    # 응답이 JSON 형식인지 시도
    try:
        data = response.json()

        # 데이터 구조 분석
        if isinstance(data, list) and len(data) >= 2:
            ds_list = data[1].get('dsList', [])
            res_header = data[1].get('resHeader', [{}])[0]

            print(f"\n응답 상태 코드: {res_header.get('SS_CODE', 'N/A')}")
            print(f"응답 시간: {res_header.get('RS_DTTM', 'N/A')}")
            print(f"조회된 데이터 개수: {len(ds_list)}개\n")

            if ds_list:
                print("조회된 공고 목록:")
                for idx, item in enumerate(ds_list, 1):
                    print(f"\n[{idx}] 공고 정보:")
                    for key, value in item.items():
                        print(f"  - {key}: {value}")
                    print("-"*60)
            else:
                print("조회된 데이터가 없습니다.")
                print("검색 조건을 변경해보세요.")

        print("\n전체 JSON 응답:")
        print(json.dumps(data, indent=2, ensure_ascii=False))

    # JSON 파싱 실패 시 (XML 응답)
    except json.JSONDecodeError:
        print("--- [원본 응답 (XML 예상)] ---")
        print(response.text)

except requests.exceptions.RequestException as e:
    print(f"API 호출 오류: {e}")

API 호출 성공

응답 상태 코드: Y
응답 시간: 20251117082150
조회된 데이터 개수: 0개

조회된 데이터가 없습니다.
검색 조건을 변경해보세요.

전체 JSON 응답:
[
  {
    "dsSch": [
      {
        "PAN_ED_DT": "20251117",
        "PG_SZ": "10",
        "PAN_ST_DT": "20250917",
        "PAGE": "1",
        "UPP_AIS_TP_CD": "01",
        "PAN_NM": "서울",
        "CNP_CD": "11",
        "PAN_SS": "공고중"
      }
    ]
  },
  {
    "dsList": [],
    "resHeader": [
      {
        "RS_DTTM": "20251117082150",
        "SS_CODE": "Y"
      }
    ]
  }
]


In [14]:
# !pip install pymupdf4llm

In [15]:
import pymupdf4llm

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [16]:
def convert_pdf_to_markdown(pdf_path, output_path=None):
    """
    PDF 파일을 Markdown으로 변환

    Args:
        pdf_path: PDF 파일 경로
        output_path: 출력 파일 경로 (기본값: PDF 파일명.md)

    Returns:
        tuple: (output_path, page_count, char_count)
    """
    # PDF를 페이지별로 Markdown 변환
    pages = pymupdf4llm.to_markdown(pdf_path, page_chunks=True)

    # 전체 텍스트 조합
    full_text = ''
    for page_data in pages:
        page_num = page_data['metadata']['page']
        page_text = page_data['text']
        full_text += f'\n=== 페이지 {page_num + 1} ===\n{page_text}'

    # 출력 파일 경로 설정
    if output_path is None:
        output_path = pdf_path.replace('.pdf', '.md')

    # Markdown 파일 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(full_text)

    return output_path, len(pages), len(full_text)

In [17]:
# downloads 폴더의 모든 PDF 파일 변환
download_path = os.path.abspath('./downloads')

# 폴더 존재 확인
if not os.path.exists(download_path):
    print(f"오류: {download_path} 폴더가 존재하지 않습니다.")
    print("downloads 폴더를 먼저 생성해주세요.")
else:
    pdf_files = glob.glob(os.path.join(download_path, '*.pdf'))

    # PDF 파일 유무 확인
    if not pdf_files:
        print("변환할 PDF 파일이 없습니다.")
        print(f"경로: {download_path}")
    else:
        print(f"변환 대상 PDF: {len(pdf_files)}개")
        print("="*60)

        conversion_results = []

        for pdf_file in pdf_files:
            try:
                print(f"\n변환 중: {os.path.basename(pdf_file)}")
                output_path, page_count, char_count = convert_pdf_to_markdown(pdf_file)

                conversion_results.append({
                    'pdf': os.path.basename(pdf_file),
                    'markdown': os.path.basename(output_path),
                    'pages': page_count,
                    'chars': char_count,
                    'success': True
                })

                print(f"  - 페이지: {page_count}개")
                print(f"  - 문자 수: {char_count:,}자")
                print(f"  - 저장: {os.path.basename(output_path)}")

            except Exception as e:
                print(f"  - 오류 발생: {e}")
                conversion_results.append({
                    'pdf': os.path.basename(pdf_file),
                    'success': False,
                    'error': str(e)
                })

        print("\n" + "="*60)
        print(f"변환 완료: {sum(1 for r in conversion_results if r['success'])}개")
        print(f"변환 실패: {sum(1 for r in conversion_results if not r['success'])}개")

변환 대상 PDF: 1개

변환 중: [공고문]부천여월LH참여형가로주택정비사업(브라운스톤여월)행복주택입주자모집(`25.11.14.).pdf
  - 페이지: 14개
  - 문자 수: 48,702자
  - 저장: [공고문]부천여월LH참여형가로주택정비사업(브라운스톤여월)행복주택입주자모집(`25.11.14.).md

변환 완료: 1개
변환 실패: 0개


In [18]:
# 변환된 Markdown 파일 목록 확인
md_files = glob.glob(os.path.join(download_path, '*.md'))

print(f"변환된 Markdown 파일 ({len(md_files)}개):")
print("="*60)

for md_file in md_files:
    file_size = os.path.getsize(md_file)
    print(f"\n파일명: {os.path.basename(md_file)}")
    print(f"  - 크기: {file_size:,} bytes")
    print(f"  - 경로: {md_file}")

변환된 Markdown 파일 (1개):

파일명: [공고문]부천여월LH참여형가로주택정비사업(브라운스톤여월)행복주택입주자모집(`25.11.14.).md
  - 크기: 105,684 bytes
  - 경로: c:\Users\diste\Documents\GitHub\SKN_19\originals\00_project\04_project\downloads\[공고문]부천여월LH참여형가로주택정비사업(브라운스톤여월)행복주택입주자모집(`25.11.14.).md


In [19]:
# !pip install langchain langchain-openai langchain-community faiss-cpu

In [20]:
from typing import TypedDict
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langgraph.graph import StateGraph, START, END

In [21]:
class State(TypedDict):
    question: str
    context: str
    answer: str
    documents: list[Document]

In [22]:
def create_vectordb(md_path):
    """
    Markdown 파일을 벡터 DB로 변환

    Args:
        md_path: Markdown 파일 경로

    Returns:
        FAISS vectorstore
    """
    # Markdown 파일 읽기
    with open(md_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )
    chunks = text_splitter.split_text(text)

    # Document 객체로 변환
    documents = [Document(page_content=chunk) for chunk in chunks]

    # 벡터 DB 생성
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

    return vectorstore

In [23]:
def retrieve_documents(state: State) -> State:
    """
    질문과 관련된 문서 검색

    Args:
        state: 현재 상태 (question 포함)

    Returns:
        업데이트된 상태 (context, documents 추가)
    """
    question = state['question']

    # 유사도 검색
    docs = vectorstore.similarity_search(question, k=3)

    # 컨텍스트 생성
    context = "\n\n".join([doc.page_content for doc in docs])

    return {
        **state,
        'context': context,
        'documents': docs
    }

In [24]:
def generate_answer(state: State) -> State:
    """
    검색된 문서를 기반으로 답변 생성

    Args:
        state: 현재 상태 (question, context 포함)

    Returns:
        업데이트된 상태 (answer 추가)
    """
    question = state['question']
    context = state['context']

    # LLM 초기화
    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    # 프롬프트 생성
    prompt = f"""다음 문서를 참고하여 질문에 답변해주세요.

문서 내용:
{context}

질문: {question}

답변: 문서 내용을 바탕으로 정확하고 간결하게 답변해주세요."""

    # 답변 생성
    response = llm.invoke(prompt)

    return {
        **state,
        'answer': response.content
    }

In [25]:
# LangGraph workflow 구성
workflow = StateGraph(State)

# 노드 추가
workflow.add_node("retrieve", retrieve_documents)
workflow.add_node('generate', generate_answer)

# 엣지 연결
workflow.add_edge(START, 'retrieve')
workflow.add_edge('retrieve', 'generate')
workflow.add_edge('generate', END)

# 그래프 컴파일
graph = workflow.compile()

In [26]:
# 벡터 DB 생성 (첫 번째 Markdown 파일 사용)
download_path = os.path.abspath('./downloads')
md_files = glob.glob(os.path.join(download_path, '*.md'))

if md_files:
    print(f"벡터 DB 생성 중: {os.path.basename(md_files[0])}")
    vectorstore = create_vectordb(md_files[0])
    print("벡터 DB 생성 완료")
else:
    print("Markdown 파일이 없습니다. PDF를 먼저 변환해주세요.")

벡터 DB 생성 중: [공고문]부천여월LH참여형가로주택정비사업(브라운스톤여월)행복주택입주자모집(`25.11.14.).md
벡터 DB 생성 완료


In [27]:
# 테스트 질문
questions = [
    "이 문서는 무엇에 관한 내용인가요?",
    "신청 자격은 무엇인가요?",
    "필요한 서류는 무엇인가요?",
    "가산점이 있나요?"
]

for question in questions:
    print(f"질문: {question}")
    print("-" * 80)

    # 그래프 실행
    result = graph.invoke({
        "question": question,
        "context": "",
        "answer": "",
        "documents": []
    })

    print(f"답변: {result['answer']}\n")
    print("=" * 80)
    print()

질문: 이 문서는 무엇에 관한 내용인가요?
--------------------------------------------------------------------------------
답변: 이 문서는 행복주택 입주자격 및 서류 제출에 관한 내용입니다. 입주자격을 검증하기 위한 소득 및 자산 기준, 필요한 제출서류, 제출 방법, 그리고 입주자격 조사 결과에 대한 소명의무 등에 대한 정보를 포함하고 있습니다.


질문: 신청 자격은 무엇인가요?
--------------------------------------------------------------------------------
답변: 신청 자격은 다음과 같습니다:

1. **청년 계층**으로 신청하는 경우.
2. 주민등록표등본상 배우자가 확인되지 않는 경우 (세대분리, 미혼, 이혼, 사별 등).
3. 배우자가 없는 경우에도 제출이 필요.
4. 신청자가 세대주가 아닌 경우 (예: 동거인) 무주택세대구성원 확인이 필요한 경우.
5. 출생자녀 또는 입양자녀가 있어 소득·자산기준을 상향하여 인정받으려는 경우.

또한, 특정 서류를 제출해야 하며, 인터넷 신청자는 서류 제출대상자로 확정된 후 제출해야 하고, 현장 신청자는 접수 시점에 제출해야 합니다.


질문: 필요한 서류는 무엇인가요?
--------------------------------------------------------------------------------
답변: 필요한 서류는 다음과 같습니다:

1. **거주지가 아닌 소득 근거지로 신청한 자**:
   - 국민연금 가입증명서 (전체이력으로 발급)
   - 건강보험자격득실확인서 (전체내역으로 발급)
   - 소득활동 중인 직장의 사업자등록증 사본 또는 법인등기부등본
   - 직장소재지 확인 불가 시 재직증명서 또는 근로계약서 등 추가 제출

2. **소득활동 중이나 국민연금 가입증명서상 '납부예외', '임의(계속)가입', '미가입자', '지역가입자'인 경우**:
   - 사업자등록증